<a href="https://colab.research.google.com/github/cheatham1/EU-JAV-Models/blob/main/Plot_EU_JAV_VaccineStance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade transformers
!pip install sentencepiece

import numpy as np
import pandas as pd
import json

import torch

from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import pipeline


     |████████████████████████████████| 3.4 MB 13.9 MB/s 
     |████████████████████████████████| 596 kB 73.1 MB/s 
     |████████████████████████████████| 895 kB 78.2 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 44.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 13.4 MB/s 


# New Section

In [7]:
model_name = "Cheatham/xlm-roberta-large-finetuned"
#model_name = "Cheatham/xlm-roberta-large-finetuned4"

In [ ]:
config = AutoConfig.from_pretrained(model_name)

#model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, config=config)

tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)


Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

In [ ]:
!ls

In [18]:
test_data = pd.read_csv(
    #dataset1 test
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/july_test_3categories_2.csv',

    #dataset2 test
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/no_replies_no_mentions_oct_test_3categories_2.csv',
    
  #  names=["label", "text"]

    # TOTAL dataset2 with dates
    'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/no_replies_no_mentions_date_oct_3categories.csv',
    names=["label", "text","YMD", "month"]
)


In [ ]:
test_data.head(2)

In [ ]:
test_data.text.iloc[1]

In [23]:
X_test = list(test_data["text"])

In [ ]:
len(X_test)

In [ ]:
test_data.head()

In [ ]:
test_data.dtypes

## EU-JAVA transformer model
## Classification of tweets for vaccine stance

In this notebook we will use a transformer model for sequence classification we have finetuned on Italian tweets


In [19]:
classifier = pipeline( task="text-classification"  , model=model, tokenizer=tokenizer, config=config)

In [ ]:
classifier(["avviso vaccino antinfluenzale 2018-19. come il mercurio causa processi neurodegenerativi cerebrali link"])

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [21]:
import time


In [24]:
t1 = time.perf_counter()

pred_label = []
#for out in tqdm(classifier(X_test)):
for out in classifier(X_test):
    pred_label.append(out)

t2 = time.perf_counter()
print('time taken to run:',t2-t1)


time taken to run: 314.3273858419998


In [ ]:
len(pred_label)

In [ ]:
pred_label[:10]

In [25]:
labels = []
for pred in pred_label:
    labels.append(pred["label"])


In [26]:
int_labels = []

for string in labels:
    int_label = string.replace("LABEL_", "")
    int_label = int(int_label)
    int_labels.append(int_label)

In [ ]:
int_labels[:10]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

In [ ]:
sns.set_palette("Blues_r")
p = sns.countplot(x=labels)
p.set_title("Dataset2: Stance Analysis Model Predictions")
p.set_xticklabels(["Promotional","Neutral","Discouraging"])

In [ ]:
X_true_labels = list(test_data["label"])

In [ ]:
X_true_labels[:10]

In [ ]:
sns.set_palette("Greens_r")
p = sns.countplot(x= X_true_labels)
p.set_title("Dataset2: Stance Analysis Labelled Data")
p.set_xticklabels(["Promotional","Neutral","Discouraging"])

In [ ]:
# Compute confusion matrix
#import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.metrics import confusion_matrix

sns.set_theme(style="white", palette=None)

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 8
myfont_size = 18

In [ ]:
confusion_matrix(int_labels, X_true_labels)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar(shrink=0.7)
    tick_marks = np.arange(len(classes))
    #plt.xticks(tick_marks, classes, rotation=45)
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",
                 fontsize=myfont_size)

    plt.tight_layout()
    plt.ylabel('True label',fontsize=myfont_size)
    plt.xlabel('Predicted label',fontsize=myfont_size)

In [ ]:
import itertools
class_names= ["Promotional","Neutral","Discouraging"]

cnf_matrix = confusion_matrix(int_labels, X_true_labels)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names)
plt.title('Confusion matrix, without normalization',fontsize=myfont_size)


# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')
plt.title('Normalized Confusion Matrix',fontsize=myfont_size)

plt.tight_layout()
#plt.savefig("ConfusionMatrix.png")
plt.show()

In [ ]:
test_data["Prediction"] = int_labels

In [ ]:
test_data.head(10)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = 15, 8
myfsize = 17


discouraging_colour= "#fc7d0b"   
neutral_colour = "#86B049"
promotional_colour = "#1170aa"

colours_3 = [promotional_colour,neutral_colour,discouraging_colour]

stance_colours_3 = np.array([promotional_colour,
                             neutral_colour,
                             discouraging_colour,      
                            ])
cmap_stance = matplotlib.colors.LinearSegmentedColormap.from_list("", stance_colours_3 )


In [ ]:
grouped_df = test_data.groupby(['month', 'label']).size().reset_index(name="Count")
sub_df = grouped_df.groupby(['month','label'])['Count'].sum().unstack()

sub_df.columns = ("promotional", "neutral", "discouraging")

In [ ]:
#plt.style.use('ggplot')
sns.set_theme(style="white", palette=None)

p = sub_df.plot(kind='bar', stacked=True, cmap=cmap_stance ) 

p.set_xticklabels(('Feb', 'Mar', 'Apr', 'May','Jun', 'Jul','Aug', 'Sep'))
p.tick_params(labelsize=myfsize)

p.set_title('EU_JAV: Stance Tweet Count After Cleaning: Feb - Sept 2021',fontsize=myfsize)

p.set_xlabel("Month 2021",fontsize=myfsize, labelpad=20)
p.set_ylabel("Number of Tweets",fontsize=myfsize)
p.legend(fontsize='x-large')

fig = p.get_figure()

In [ ]:
grouped_df_2 = test_data.groupby(['month', 'Prediction']).size().reset_index(name="Count")
sub_df_2 = grouped_df_2.groupby(['month','Prediction'])['Count'].sum().unstack()

sub_df_2.columns = ("promotional", "neutral", "discouraging")

In [ ]:
p_2 = sub_df_2.plot(kind='bar', stacked=True, cmap=cmap_stance ) 

p_2.set_xticklabels(('Feb', 'Mar', 'Apr', 'May','Jun', 'Jul','Aug', 'Sep'))
p_2.tick_params(labelsize=myfsize)

p_2.set_title('EU_JAV: Stance Prediction: Feb - Sept 2021',fontsize=myfsize)

p_2.set_xlabel("Month",fontsize=myfsize, labelpad=20)
p_2.set_ylabel("Number of Tweets",fontsize=myfsize)
p_2.legend(fontsize='x-large')

fig = p.get_figure()
fig = p_2.get_figure()

In [ ]:
sub_df

In [ ]:
sub_df.columns

In [ ]:
sub_df_2

In [ ]:
fig, ax = plt.subplots()

#ax.plot(sub_df)
#sub_df_2.plot(ax=ax)


ax.plot(sub_df)
sub_df_2.plot(ax=ax)


plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(sub_df, linewidth=4 )
ax.plot(sub_df_2, linewidth=2, linestyle='dashed')


plt.show()

In [ ]:

ax = sub_df.plot(kind='bar', cmap=cmap_stance ) 
sub_df_2.plot(ax= ax, kind='line', cmap=cmap_stance ) 

#plt.show()
